In [ ]:
#import libraries
from pathlib import Path
import os
import napari
from napari.utils.notebook_display import nbscreenshot
import numpy as np
from skimage.io import imread, imshow, imsave 
import glob
from cellpose import models, io

In [ ]:
'''Uncommnent this cells if your images are .Bmp. and  need to be converted to .tiff (or .tif) to be used in the Cellpose GUI '''
# #code for converting .Bmp files to .tiff files
# tiff_files = np.sort(glob.glob('images/*Channel 2*.Bmp'))
# output_folder = 'images_and_model/'
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
# for i in tiff_files:
#     image = imread(i)
#     output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(i))[0]}.tiff")
#     imsave(output_path, image)
    

In [ ]:
model = models.CellposeModel(gpu=True, pretrained_model = 'path_to/images_and_model')  # if no GPU, set gpu=False
channels = [0,0]
flow_threshold = 0.8



tiff_files = np.sort(glob.glob('images_and_model/*Channel*.tiff'))


output_folder = 'cellpose_label_images/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for img_path in tiff_files:
    print("Processing image:", img_path)
    
    try:
        image = imread(img_path)
        #run cellpose
        label_image, flows, styles = model.eval(image, flow_threshold=flow_threshold, 
                                               channels=channels)
        output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(img_path))[0]}_cellpose.tiff")
        imsave(output_path, label_image)
        print(f"Saved label to: {output_path}")
        
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
# For loop for running nuclear segmentation, generation of eroded and ring labels and quantification

   
    
    
    

   
        



In [ ]:
dataset_path = Path('images_and_model/')
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(dataset_path.glob("*Channel*.tiff")), stack=True)
image = viewer.layers[0].data
viewer.add_image(image, visible=True)

In [ ]:
dataset_path = Path('cellpose_label_images/')
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(dataset_path.glob("*Channel*.tiff")), stack=True)
cellpose_label = viewer.layers[0].data
viewer.add_labels(cellpose_label, visible=True)
viewer.add_image(image, visible=True)